# 2D toy model

- start from segmented 2d image of cells
- modify image to create explicit membrane compartments
- combine cells, combine membranes
- create a simple sme model using this geometry
- do an example simulation

## Navigation

- Press `Space` to show the next page
- Press `Shift+Space` to show the previous page
- Press `Escape` to zoom out

## Input data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
import imageio.v3 as iio
import skimage
import sme
from IPython.display import HTML
from scipy import ndimage as ndi

plt.rcParams["figure.figsize"] = (8, 8)


def show_image(img, title=""):
    plt.imshow(img, interpolation="none")
    plt.title(title)
    plt.show()

In [ ]:
show_image(
    iio.imread("C2-sph1_gly_au-0036_segmentedWithCellpose.png"),
    "Segmentation overlayed on source image",
)

In [ ]:
img_indexed = iio.imread("C2-sph1_gly_au-0036_cp_masks.png")
show_image(skimage.color.label2rgb(img_indexed), "Segmented image")

### Generate explicit membranes by dilating each cell

- We want to add explicit membrane compartments around each cell.
- To do this we take a mask of each cell individually, dilate it, and select the pixels that differ from the original mask
- Repeating this over all cells and combining the results gives us a mask of membrane compartment pixels

In [ ]:
img_membrane_mask = np.zeros(img_indexed.shape).astype(bool)
for index in range(img_indexed.max()):
    img = (img_indexed == index).astype(np.uint8)
    img_membrane_mask |= ndi.binary_dilation(img, iterations=3) != img

In [ ]:
show_image(img_membrane_mask, "Membrane pixels mask")

### Define cells as any segmented pixel excluding membrane pixels

- Now we select all pixels that were identified as cells
- Then we exclude pixels that are part of the membrane mask to leave a cell mask

In [ ]:
img_cell_mask = img_indexed != 0
img_cell_mask = img_cell_mask & (img_cell_mask != img_membrane_mask)

In [ ]:
show_image(img_cell_mask, "Cell pixels mask")

### Construct segmented geometry image for sme

- From these masks we can construct a segmented geometry image for sme
- Each colour in this image can then be assigned to a compartment in the model

In [ ]:
img = np.zeros((*img_cell_mask.shape, 3), dtype=np.uint8)
img[img_cell_mask] = [100, 100, 255]
img[img_membrane_mask] = [255, 100, 100]
iio.imwrite("geom.png", img)

In [ ]:
show_image(img, "Segmented geometry image for sme")

### Create sme model

- This was done using the GUI
- Geometry image was downscaled to 512x512 pixels
- One species in each compartment, intially only non-zero in outside
- Reactions: `outside <-> membrane` and `membrane <-> cell`

In [ ]:
model = sme.Model("2d-toy-model.xml")
show_image(model.compartment_image[0, :])

### Simulate model

- simulate for 20000s, storing the results every 100s
- this might take a few minutes

In [ ]:
simulation_results = model.simulate(20000, 100)

### Create animation of simulation results

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)
ax.set_title(f"Concentration")
norm = plt.Normalize(vmin=0, vmax=1)
im = ax.imshow(np.zeros((1, 1)), norm=norm)
fig.colorbar(im, ax=ax)
artists = [
    [
        ax.imshow(
            sum(simulation_result.species_concentration.values())[0, :],
            animated=True,
            interpolation=None,
            norm=norm,
        )
    ]
    for simulation_result in simulation_results
]
anim = animation.ArtistAnimation(fig, artists, interval=200, blit=True, repeat=False)
plt.close()

In [ ]:
HTML(anim.to_html5_video())